In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widjets
%matplotlib inline

import numpy as np
from torchvision import datasets, models, transforms
import torch
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
import pandas as pd

import matplotlib.pyplot as plt

import os
import shutil
import cv2
from copy import deepcopy

from sklearn.model_selection import train_test_split
import itertools
import matplotlib.pyplot as plt

from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from time import time
from tqdm import tqdm, tqdm_notebook

from sklearn.metrics import precision_score, recall_score, roc_auc_score, confusion_matrix
from sklearn import metrics
import json
from pathlib import Path

In [4]:
import sys
sys.path.append(r'Z:\test tasks\Sber_robo\project')

In [19]:
from project.model import my_segmentator
model,preprocessing_fn = my_segmentator()
model.load_state_dict(torch.load(r'Z:\test tasks\Sber_robo\project\logs\checkpoints\last_full.pth')['model_state_dict'])
model.eval().cuda()
0

0

In [20]:
import albumentations as albu
import cv2
from albumentations.pytorch import ToTensor

BORDER_CONSTANT = 0
def pre_transforms(image_size):
    return albu.Compose([
        albu.LongestMaxSize(max_size=image_size, always_apply=True),
        albu.PadIfNeeded(min_height=None, min_width=None, pad_height_divisor=32, pad_width_divisor=32, border_mode=BORDER_CONSTANT, value=0),
    ])

def get_preprocessing(preprocessing_fn):
    return albu.Compose([
        albu.Lambda(name='my_transform', image=preprocessing_fn),
        ToTensor(),
    ])

valid_transforms = pre_transforms
preprocess_tr = get_preprocessing(preprocessing_fn)

In [22]:
from dataset_SDD import SDD_dataset, CLASSES
from dataset_UAVid import UAVid_dataset

SDD_val_PATH = r'Z:\test tasks\Sber_robo\SDD_images\imgs\test'

valid_transforms = pre_transforms(image_size=512)
SDD_val = SDD_dataset(SDD_val_PATH, augmentations=valid_transforms, preprocessing=None)

#############
UAVid_val_PATH = r'Z:\test tasks\Sber_robo\uavid_v1.5_official_release_image\uavid_val'

valid_transforms = pre_transforms(image_size=1024)
UAVid_val = UAVid_dataset(UAVid_val_PATH, augmentations=valid_transforms, preprocessing=None)

found 50 images


In [28]:
CLASSES = {'restricted_area': 0, 'road': 1, 'building': 2, 'human': 3, 'car': 4}
CLASSES = CLASSES.keys()
DEFAULT_COLORS = ('red', 'blue', 'yellow', 'magenta', 'green', 'indigo', 'darkorange', 'cyan', 'pink', 'yellowgreen')

import matplotlib.patches as mpatches
import matplotlib.pyplot as plt

patches = []
for i, cls in enumerate(CLASSES):
    patches.append(mpatches.Patch(color=DEFAULT_COLORS[i], label=cls))

def mask_to_overlay_image(image, labels, mask_strength=0.5):
    from skimage.color import label2rgb
    
    mask = label2rgb(labels, bg_label=-1) 

    image = np.array(image) / 255.0
    image_with_overlay = image * (1 - mask_strength) + mask * mask_strength
    image_with_overlay = (
        (image_with_overlay * 255).clip(0, 255).round().astype(np.uint8)
    )

    return image_with_overlay

In [26]:
images_paths = Path(r'Q:\Downloads\stanford_campus_dataset\annotations').glob('*/*/reference.jpg')
valid_transforms = pre_transforms(image_size=512)
Inference_images = []
Inference_masks = []
with torch.no_grad():
    for img_p in tqdm_notebook(list(images_paths)):
        img = cv2.imread(str(img_p))[:,:,::-1]
        img = valid_transforms(image=img)['image']
        
        
        tensor_img = preprocess_tr(image=img)['image'].unsqueeze(0)
        preds = model(tensor_img.cuda())
        
        Inference_images.append(img)
        Inference_masks.append(torch.argmax(preds[0], dim=0).cpu().detach().numpy())

c:\python37\lib\site-packages\ipykernel_launcher.py:6: TqdmDeprecationWarning:

This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



In [29]:
@interact(i=(0,len(Inference_images)-1,1))
def tst(i=0):
    img = Inference_images[i]
    masks = Inference_masks[i]
    
    plt.figure(figsize=(15,20))
    plt.subplot(1,2,1)
    plt.imshow(img)
    
    plt.subplot(1,2,2)
    
    a = mask_to_overlay_image(img, masks)
    plt.legend(handles=patches, ncol=7, bbox_to_anchor=(-0.1, 1),
              loc='lower left', fontsize='small')
    plt.imshow(a)


interactive(children=(IntSlider(value=0, description='i', max=59), Output()), _dom_classes=('widget-interact',…